# Data Loading & Cleaning (Fixed)

This notebook addresses the data quality issues found in `train_raw.csv`.
The key steps are:
1.  **Load Data**: Read the raw CSV file.
2.  **Initial Cleanup**: Fix column names and apply initial data types.
3.  **Handle Categorical Features**: Clean up and encode categorical columns like `occupation`, `credit_mix`, and `payment_of_min_amount`.
4.  **Handle Missing Values (NaNs)**: Systematically impute missing values for both numeric and categorical columns.
5.  **Handle Outliers**: Cap extreme values in numeric columns to reduce their skewness.
6.  **Feature Transformation**: Convert columns like `credit_history_age` to a numeric format.
7.  **Drop Unnecessary Columns**: Remove identifiers that are not useful for modeling.
8.  **Save Cleaned Data**: Export the processed DataFrame to a new CSV file.

In [1]:
import pandas as pd
import numpy as np
import re

## 1. Load Data and Initial Schema Application

In [2]:
file_name = "train_raw.csv"

# Define the schema for data types
string_cols = [
    "customer_id", "month", "name", "ssn",
    "occupation", "type_of_loan", "credit_history_age",
    "credit_mix", "payment_of_min_amount", "payment_behaviour", "credit_score"
]

# Read raw CSV without strict dtype enforcement for numeric columns
df = pd.read_csv(file_name, na_values=["", "NA", "N/A", "NaN", "_______", "_", "!@9#%8"])

# Convert column names to lower case
df.columns = df.columns.str.lower()

# Coerce numeric columns, turning errors into NaN
for col in df.columns:
    if col not in string_cols:
        # .str.replace(',', '') - converts numbers 1,200 to 1200
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors="coerce")

# Create a unique row_id and drop original identifiers
if "id" in df.columns:
    df["row_id"] = df.index
    df = df.drop(columns=["id", "ssn", "name"], errors="ignore")

df.head()

/tmp/ipykernel_34045/2549237833.py:11: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name, na_values=["", "NA", "N/A", "NaN", "_______", "_", "!@9#%8"])


,customer_id,month,age,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,...,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score,row_id
0,CUS_0xd40,January,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,...,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good,0
1,CUS_0xd40,February,23.0,Scientist,19114.12,NaN,3,4,3,4.0,...,809.98,31.944960,NaN,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good,1
2,CUS_0xd40,March,-500.0,Scientist,19114.12,NaN,3,4,3,4.0,...,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good,2
3,CUS_0xd40,April,23.0,Scientist,19114.12,NaN,3,4,3,4.0,...,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,Good,3
4,CUS_0xd40,May,23.0,Scientist,19114.12,1824.843333,3,4,3,4.0,...,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,Good,4


### Simplify the dataset
**The issue**: since customers have several rows, it is a time-series prediction problem to predict `credit_score` in the next month.

Let's simplify the dataset by keep the latest month per customer (most recent data = best predictor)

In [3]:
# Convert month to numeric for sorting
month_map = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4,
    'May': 5, 'June': 6, 'July': 7, 'August': 8,
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}
df['month_num'] = df['month'].map(month_map)

# Sort by customer and month (ascending)
df_sorted = df.sort_values(['customer_id', 'month_num'])

# Keep ONLY the last month for each customer
df = df_sorted.groupby('customer_id', as_index=False).last()

## 2. Clean Categorical and String Columns

In [4]:
# Clean 'occupation'
df['occupation'] = df['occupation'].replace('_______', 'Unknown')
df['occupation'] = df['occupation'].fillna('Unknown')

# Clean 'credit_mix'
df['credit_mix'] = df['credit_mix'].replace('_', 'Unknown')
df['credit_mix'] = df['credit_mix'].fillna('Unknown')

# Clean 'payment_of_min_amount'
df['payment_of_min_amount'] = df['payment_of_min_amount'].replace('NM', pd.NA)
# Impute with mode
mode_payment = df['payment_of_min_amount'].mode()[0]
df['payment_of_min_amount'] = df['payment_of_min_amount'].fillna(mode_payment)

# Clean 'payment_behaviour'
df['payment_behaviour'] = df['payment_behaviour'].replace('!@9#%8', pd.NA)
mode_behaviour = df['payment_behaviour'].mode()[0]
df['payment_behaviour'] = df['payment_behaviour'].fillna(mode_behaviour)

# One-hot encode 'credit_mix' and 'payment_of_min_amount'
df = pd.get_dummies(df, columns=['credit_mix', 'payment_of_min_amount'], prefix=['credit_mix', 'payment_min'], dummy_na=False)

# One-hot encode 'payment_behaviour'
df = pd.get_dummies(df, columns=['payment_behaviour'], prefix=['pay_beh'], dummy_na=False)

# One-hot encode 'occupation'
df = pd.get_dummies(df, columns=['occupation'], prefix=['occ'], dummy_na=False)

## 3. Clean and Transform `type_of_loan`

In [5]:
# Extract unique loan types
loan_types = set()
for val in df["type_of_loan"].dropna():
    normalized = val.replace(" and", ", ")
    types = [t.strip() for t in normalized.split(",") if t.strip()]
    loan_types.update(types)

# Build all new columns as dict
loan_data = {}
for loan in sorted(loan_types):
    column_name = f"has_{loan.lower().replace(' ', '_').replace('-', '_')}"
    loan_data[column_name] = df["type_of_loan"].apply(
        lambda s, loan=loan: 1 if pd.notna(s) and loan in s else 0
    ).astype("int8")

# Create DataFrame once
loan_dummies = pd.DataFrame(loan_data, index=df.index)

# Merge and clean
df = pd.concat([df.drop(columns=["type_of_loan"], errors="ignore"), loan_dummies], axis=1).copy()

## 4. Transform `credit_history_age`

In [6]:
from typing import Optional


def convert_string_to_months(s: str) -> Optional[int]:
    if pd.isna(s):
        return pd.NA
    try:
        # Matches "X Years and Y Months"
        match = re.search(r"(\d+)\s*Years\s*and\s*(\d+)\s*Months", s)
        if match:
            years = int(match.group(1))
            months = int(match.group(2))
            return years * 12 + months
    except (ValueError, AttributeError):
        return pd.NA
    return pd.NA

df["credit_history_months"] = df["credit_history_age"].apply(convert_string_to_months).astype('Int64')
median_history_months = df["credit_history_months"].median()
df["credit_history_months"] = df["credit_history_months"].fillna(median_history_months)
df = df.drop(columns=["credit_history_age"], errors="ignore")

## 5. Handle Outliers and Impute Missing Numeric Values

Here we will address the numeric columns with outliers and missing values.
The strategy is:
1.  **Clip/Cap**: For columns with extreme outliers, we cap the values at a reasonable percentile (e.g., 99th) or a logical maximum.
2.  **Impute**: We fill missing values (`NaN`) using the median, which is robust to outliers.

In [7]:
numeric_cols_to_process = {
    'annual_income': (0, 0.99),
    'monthly_inhand_salary': (0, 0.99),
    'num_bank_accounts': (0, 25), # Cap at a reasonable number
    'num_credit_card': (0, 25),   # Cap at a reasonable number
    'num_of_loan': (0, 25),        # Cap at a reasonable number
    'num_credit_inquiries': (0, 0.99),
    'changed_credit_limit': (None, 0.99), # Allow negative, cap upper
    'outstanding_debt': (0, None),
    'amount_invested_monthly': (0, 0.99),
    'total_emi_per_month': (0, 0.99),
    'age': (18, 70), # Clip to a reasonable age range
    'num_of_delayed_payment': (0, 0.99),
    'delay_from_due_date': (None, None), # No obvious outliers to cap
    'interest_rate': (0, 0.99), # Cap extreme outliers
    'monthly_balance': (None, None) # Impute NaNs
}

for col, (lower_bound, upper_bound) in numeric_cols_to_process.items():
    if col in df.columns:
        # Cap outliers
        if lower_bound is not None:
            if isinstance(lower_bound, float):
                lower_quantile = df[col].quantile(lower_bound)
                df[col] = df[col].clip(lower=lower_quantile)
            else:
                df[col] = df[col].clip(lower=lower_bound)

        if upper_bound is not None:
            if isinstance(upper_bound, float):
                upper_quantile = df[col].quantile(upper_bound)
                df[col] = df[col].clip(upper=upper_quantile)
            else:
                 df[col] = df[col].clip(upper=upper_bound)

        # Impute missing values with the median
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)

# Convert integer-like columns to int
for col in ['num_bank_accounts', 'num_credit_card', 'num_of_loan', 'num_of_delayed_payment', 'age']:
    if col in df.columns:
        df[col] = df[col].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12500 entries, 0 to 12499
Data columns (total 57 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   customer_id                               12500 non-null  object 
 1   month                                     12500 non-null  object 
 2   age                                       12500 non-null  int64  
 3   annual_income                             12500 non-null  float64
 4   monthly_inhand_salary                     12500 non-null  float64
 5   num_bank_accounts                         12500 non-null  int64  
 6   num_credit_card                           12500 non-null  int64  
 7   interest_rate                             12500 non-null  float64
 8   num_of_loan                               12500 non-null  int64  
 9   delay_from_due_date                       12500 non-null  int64  
 10  num_of_delayed_payment            

## 6. Final Review and Save

In [8]:
# Display summary statistics for the cleaned data
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,12500.0,34.23144,11.562668,18.0,25.0,33.0,42.0,70.0
annual_income,12500.0,51726.17876,40094.045819,7005.93,19454.9725,37578.215,72692.745,179068.0832
monthly_inhand_salary,12500.0,4182.130403,3159.711186,303.645417,1624.937917,3087.595,5947.364167,13789.2362
num_bank_accounts,12500.0,5.62664,3.425344,0.0,3.0,6.0,7.0,25.0
num_credit_card,12500.0,5.99584,3.596695,0.0,4.0,5.0,7.0,25.0
interest_rate,12500.0,58.5401,329.066462,1.0,8.0,14.0,20.0,2865.01
num_of_loan,12500.0,3.49352,2.943048,0.0,1.0,3.0,5.0,25.0
delay_from_due_date,12500.0,21.06088,14.863091,-5.0,10.0,18.0,28.0,67.0
num_of_delayed_payment,12500.0,13.44288,6.371554,0.0,9.0,14.0,18.0,27.0
changed_credit_limit,12500.0,10.376779,6.76617,-6.49,5.35,9.4,14.9125,28.8001


#### Remove the `customer_id` column

In [9]:
df = df.drop(["customer_id"], errors="ignore")

In [10]:
# Drop the unique identifier before saving
df = df.drop(columns=['row_id'], errors='ignore')

# Save the cleaned dataframe
df.to_csv("train_cleaned.csv", index=False)

print("Cleaned data saved to train_cleaned.csv")

Cleaned data saved to train_cleaned.csv
